# Vertical Bin Packing Problem

- Airport and Cargo Operations (AE4446)
- Zach Kelly (5901405), Timotei Dudas ()
- May 5th, 2023

### Imports

In [3]:
import pandas as pd
import numpy as np 
import gurobipy as gp
from gurobipy import Model, GRB

### Input Data

In [4]:
# load data 
ulds = pd.read_pickle('../data/B.pickle')
items = pd.read_pickle('../data/R.pickle')

print('----- ULDs -----')
display(ulds)
print('----- Items -----')
display(items)

----- ULDs -----


{0: (0, [300, 155, 2, 140, 0, 0, 0]),
 1: (0, [300, 155, 2, 140, 0, 0, 0]),
 2: (1, [192, 155, 2, 200, 0, 0, 0]),
 3: (1, [192, 155, 2, 200, 0, 0, 0])}

----- Items -----


{0: (98, 50, 1, 0, 0, 0),
 1: (89, 46, 1, 0, 0, 0),
 2: (64, 24, 1, 0, 0, 0),
 3: (93, 29, 1, 1, 0, 0),
 4: (114, 54, 1, 1, 0, 0),
 5: (95, 60, 1, 0, 0, 0),
 6: (63, 28, 1, 0, 0, 0),
 7: (100, 46, 0, 0, 0, 0),
 8: (52, 61, 1, 0, 0, 0),
 9: (45, 46, 1, 0, 0, 0),
 10: (111, 32, 1, 0, 0, 0),
 11: (109, 38, 1, 0, 1, 0),
 12: (97, 57, 1, 0, 0, 0),
 13: (51, 29, 1, 0, 0, 0),
 14: (86, 54, 1, 0, 0, 0),
 15: (114, 31, 1, 1, 0, 0),
 16: (81, 47, 1, 0, 0, 0),
 17: (78, 25, 1, 0, 0, 0),
 18: (78, 44, 1, 0, 0, 0),
 19: (68, 33, 1, 0, 0, 0),
 20: (51, 45, 1, 0, 0, 0),
 21: (84, 36, 1, 0, 0, 1),
 22: (66, 35, 0, 0, 0, 0),
 23: (68, 42, 1, 1, 0, 0),
 24: (108, 57, 1, 1, 1, 0)}

### Setup Model

In [5]:
print('Creating Model')
print('--------------------')
model = Model()

Creating Model
--------------------
Restricted license - for non-production use only - expires 2024-10-28


### Model Parameters 

In [13]:
# total number of boxes to be packed
n = len(items.keys())
# total number of available ULDs
m = len(ulds.keys())
# dimensions of items
li = [items[i][0] for i in range(n)] # length
hi = [items[i][1] for i in range(n)] # height
# dimensions of ULDs 
lc = [300,192]
hc = [155,155]
# maximum area of ULDs
ac = [lc[j]*hc[j] for j in range(len(lc))]

# find largest uld length and width
Lj_max = np.max(np.array(list(dict(ulds.values()).values()))[:,0])
Hj_max = np.max(np.array(list(dict(ulds.values()).values()))[:,1])

300

155

### Model Variables

In [15]:
print('Creating Decision Variables')
print('--------------------')

### initialize decision variables ###
# if item i is in ULD j (binary)
p = {}
# if container j is used (binary)
u = {}
# if item i is to the right of item k (binary)
x = {}
# if item i is above item k (binary)
z = {}
# orientation of item i w.r.t uld
r = {}
# location of item i
xi = {}
zi = {}

'''
# location of lower left corner of item i 
x1 = {}
z1 = {}
# location of upper right corner of item i
x2 = {}
z2 = {}
'''

### defining the possible inputs to our decision variables while defining the actual variable type itself ###
# creating x and z decision variables
# domain : all matches between different items
for i in items.keys():
    for k in items.keys():
        if k != i:
            x[i,k]=model.addVar(lb=0, ub=1, vtype=GRB.BINARY,name="x[%s,%s]"%(i,k))
            z[i,k]=model.addVar(lb=0, ub=1, vtype=GRB.BINARY,name="z[%s,%s]"%(i,k))

# creating p decision variable
# domain : all matches between available items and bins
for i in items.keys():
    for j in ulds.keys():
        p[i,j]=model.addVar(lb=0, ub=1, vtype=GRB.BINARY,name="p[%s,%s]"%(i,j))

# creating xi and zi decision variables
for i in items.keys():
    x[i]=model.addVar(lb=0, ub=Lj_max, vtype=GRB.CONTINUOUS,name="x[%s]"%(i))
    z[i]=model.addVar(lb=0, ub=Hj_max, vtype=GRB.CONTINUOUS,name="z[%s]"%(i))

# creating r decision variable
for i in items.keys():
    for aa in range(0,2):
        for bb in range (0,2):
            r[i,aa,bb]=model.addVar(lb=0, ub=1, vtype=GRB.BINARY,name="r[%s,%s,%s]"%(i,aa,bb))

model.update()     

Creating Decision Variables
--------------------


### Model Constraints